## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1990. Count the Number of Experiments (Medium)**

**Table: Experiments**

| Column Name     | Type |
|-----------------|------|
| experiment_id   | int  |
| platform        | enum |
| experiment_name | enum |

experiment_id is the column with unique values for this table.
platform is an enum (category) type of values ('Android', 'IOS', 'Web').
experiment_name is an enum (category) type of values ('Reading', 'Sports', 'Programming').
This table contains information about the ID of an experiment done with a random person, the platform used to do the experiment, and the name of the experiment.
 
**Write a solution to report the number of experiments done on each of the three platforms for each of the three given experiments. Notice that all the pairs of (platform, experiment) should be included in the output including the pairs with zero experiments.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:**

**Experiments table:**

| experiment_id | platform | experiment_name |
|---------------|----------|-----------------|
| 4             | IOS      | Programming     |
| 13            | IOS      | Sports          |
| 14            | Android  | Reading         |
| 8             | Web      | Reading         |
| 12            | Web      | Reading         |
| 18            | Web      | Programming     |

**Output:** 
| platform | experiment_name | num_experiments |
|----------|-----------------|-----------------|
| Android  | Reading         | 1               |
| Android  | Sports          | 0               |
| Android  | Programming     | 0               |
| IOS      | Reading         | 0               |
| IOS      | Sports          | 1               |
| IOS      | Programming     | 1               |
| Web      | Reading         | 2               |
| Web      | Sports          | 0               |
| Web      | Programming     | 1               |

**Explanation:** 
- On the platform "Android", we had only one "Reading" experiment.
- On the platform "IOS", we had one "Sports" experiment and one "Programming" experiment.
- On the platform "Web", we had two "Reading" experiments and one "Programming" experiment.

In [0]:
experiments_data_1990 = [
    (4, "IOS", "Programming"),
    (13, "IOS", "Sports"),
    (14, "Android", "Reading"),
    (8, "Web", "Reading"),
    (12, "Web", "Reading"),
    (18, "Web", "Programming"),
]

experiments_columns_1990 = ["experiment_id", "platform", "experiment_name"]
experiments_df_1990 = spark.createDataFrame(experiments_data_1990, experiments_columns_1990)
experiments_df_1990.show()


+-------------+--------+---------------+
|experiment_id|platform|experiment_name|
+-------------+--------+---------------+
|            4|     IOS|    Programming|
|           13|     IOS|         Sports|
|           14| Android|        Reading|
|            8|     Web|        Reading|
|           12|     Web|        Reading|
|           18|     Web|    Programming|
+-------------+--------+---------------+



In [0]:
platforms_df_1990 = experiments_df_1990.select("platform").distinct()

In [0]:
experiments_name_df_1990 = experiments_df_1990.select("experiment_name").distinct()

In [0]:
all_combinations_1990 = platforms_df_1990.crossJoin(experiments_name_df_1990)

In [0]:
counts_df_1990 = experiments_df_1990\
                        .groupBy("platform", "experiment_name") \
                            .count() \
                                .withColumnRenamed("count", "num_experiments")

In [0]:
all_combinations_1990\
    .join(counts_df_1990, on=["platform", "experiment_name"], how="left") \
        .fillna(0, subset=["num_experiments"]).show()

+--------+---------------+---------------+
|platform|experiment_name|num_experiments|
+--------+---------------+---------------+
|     IOS|    Programming|              1|
| Android|         Sports|              0|
|     Web|        Reading|              2|
| Android|    Programming|              0|
|     Web|         Sports|              0|
|     IOS|        Reading|              0|
|     Web|    Programming|              1|
|     IOS|         Sports|              1|
| Android|        Reading|              1|
+--------+---------------+---------------+

